In [0]:
# Parameters
dbutils.widgets.text("input_file_name", "")

# Read input file
input_path = dbutils.widgets.get("input_file_name")
raw_df = spark.read.parquet(input_path)

# Add metadata columns
from pyspark.sql.functions import current_timestamp, input_file_name
bronze_df = raw_df.withColumn("input_file_name", input_file_name()) \
                  .withColumn("ingestion_timestamp", current_timestamp())

# Write to Delta with merge
bronze_df.createOrReplaceTempView("updates")

spark.sql("""
  MERGE INTO bronze_nyc_taxi target
  USING updates source
  ON source.VendorID = target.VendorID 
    AND source.tpep_pickup_datetime = target.tpep_pickup_datetime
    AND source.tpep_dropoff_datetime = target.tpep_dropoff_datetime
  WHEN NOT MATCHED THEN INSERT *
""")